In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
url ='https://www.latamairlines.com/ec/es/ofertas-vuelos/?origin=UIO&outbound=2023-12-01T12%3A00%3A00.000Z&destination=MAD&inbound=null&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=ARRIVAL_DATE%2Casc'

In [3]:
r = requests.get(url)
r.status_code

403

In [5]:
def obtener_precios(vuelo):
    '''
    Función que retorna una lista de diccionarios con las distintas tarifas
    '''
    tarifas = WebDriverWait(vuelo, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[contains(@id, "WrapperBundleCardbundle-detail-")]'))
    )
    precios = []  # crea una lista vacía
    for tarifa in tarifas:
        nombre = WebDriverWait(tarifa, 10).until(
            EC.presence_of_element_located((By.XPATH, './/div/div[1]/span'))
        ).text
        moneda = WebDriverWait(tarifa, 10).until(
            EC.presence_of_element_located((By.XPATH, './/div/div[3]/div[1]/div/div/div/div/span[1]/span[1]'))
        ).text
        valor = WebDriverWait(tarifa, 10).until(
            EC.presence_of_element_located((By.XPATH, './/div/div[3]/div[1]/div/div/div/div/span[1]/span[2]'))
        ).text
        dict_tarifa = {nombre: {'moneda': moneda, 'valor': valor}}  # arma diccionario que junta la información
        precios.append(dict_tarifa)  # para cada tarifa agrega diccionario a la lista vacía
        print(dict_tarifa)
    return precios


In [18]:
def obtener_datos_escalas(vuelo):
    '''
    Funcion que retorna una lista de diccionarios con la información de las escalas de cada vuelo
    '''
    segmentos = WebDriverWait(vuelo, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//section[@data-test="section-info-leg" and contains(@class, "itinerarystyle__Section-sc__sc-1n97ky6-1")]'))
    )
    info_escalas = []
    
    for segmento in segmentos:
        # origen
        origen = WebDriverWait(segmento, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="itinerary-modal-0-dialog-content"]/article/div/section[1]/div[1]/div/div[1]/div/span[1]'))
        ).text
        # Hora de salida
        dep_time = WebDriverWait(segmento, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="itinerary-modal-0-dialog-content"]/article/div/section[1]/div[1]/div/div[1]/div/span[2]'))
        ).text
        # Destino
        destino = WebDriverWait(segmento, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="itinerary-modal-0-dialog-content"]/article/div/section[1]/div[1]/div/div[3]/div/span[1]'))
        ).text
        # Hora de llegada
        arr_time = WebDriverWait(segmento, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="itinerary-modal-0-dialog-content"]/article/div/section[1]/div[1]/div/div[3]/div/span[2]'))
        ).text

        # Duración del vuelo
        duracion_vuelo = WebDriverWait(segmento, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="itinerary-modal-0-dialog-content"]/article/div/section[1]/div/[1]/div/div[2]/span[2]'))
        ).text
        # Numero de vuelo
        numero_vuelo = WebDriverWait(segmento, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="itinerary-modal-0-dialog-content"]/article/div/section[1]/div[3]/div/div/div/div'))
        ).text
        # Modelo del avión
        modelo_avion = WebDriverWait(segmento, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="itinerary-modal-0-dialog-content"]/article/div/section[1]/div/3]/div/div/span[1]'))
        ).text
        
        if segmento != segmentos[-1]:
            # Duración de la escala
            duracion_escala = WebDriverWait(segmento, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="itinerary-modal-0-dialog-content"]/article/div/section[2]/section/div[2]/div/div/span'))
            ).text
        else:
            duracion_escala = ''
            
        data_dict = {
            'origen': origen,
            'dep_time': dep_time,
            'destino': destino,
            'arr_time': arr_time,
            'duracion_vuelo': duracion_vuelo,
            'numero_vuelo': numero_vuelo,
            'modelo_avion': modelo_avion,
            'duracion_escala': duracion_escala,
        }
        info_escalas.append(data_dict)
        
    return info_escalas



In [14]:
def obtener_tiempos(vuelo):
    '''
    Función que retorna un lista de diccionarios con los horarios de salida y llegada de cada vuelo, incluyendo la duración.
    Nota: La diración del vuelo no es la hora de llegada - la hora de salida porque puede haver diferencia de horarios entre el 
    origen y el destino
    '''
    # Hora de salida
    salida = vuelo.find_element(By.XPATH, '//*[@id="WrapperCardFlight0"]/div/div[1]/div[2]/div[1]/div[1]/span[1]').text
    # Hora de llegada
    llegada = vuelo.find_element(By.XPATH,'//*[@id="WrapperCardFlight0"]/div/div[1]/div[2]/div[1]/div[3]/span[1]').text
    # duracion
    duracion = vuelo.find_element(By.XPATH,'//*[@id="ContainerFlightInfo0"]/span[2]').text
    tiempo = {'hora_salida':salida, 'hora_llegada':llegada, 'duracion':duracion}
    return tiempo

In [10]:
def obtener_info(browser):
    vuelos = browser.find_elements(By.XPATH, '//*[@id="CenterWrapperBodyFlights"]/ol/li')
    print(f'Se encontraron {len(vuelos)} vuelos.')
    print('Iniciando Scraping...')
    info = []

    for vuelo in vuelos:
        # Obtenemos los tiempos generales de cada vuelo
        tiempos = obtener_tiempos(vuelo)

        # Clickear sobre el botón escalas
        boton_escalas = WebDriverWait(vuelo, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="itinerary-modal-0-dialog-open"]/span'))
        )
        boton_escalas.click()

        # Obtener información de las escalas
        escalas = obtener_datos_escalas(vuelo)

        # Cerrar el modal
        cerrar_ven_modal = WebDriverWait(vuelo, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="itinerary-modal-0-dialog-close"]/span/i'))
        )
        cerrar_ven_modal.click()

        # Clickear vuelo para ver los precios
        vuelo.click()

        # Obtener los precios
        precios = obtener_precios(vuelo)

        # Cerrar precios
        vuelo.click()

        # Agregar toda la información a info
        info.append({'precios': precios, 'tiempos': tiempos, 'escalas': escalas})

    return info


In [21]:
# 1: Instanciar el navegador
# Configuración del navegador
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--incognito")
chromedriver_path = '/home/alexpino/repo1/webScraping/chromedriver/stable/chromedriver'
webdriver_service = Service(chromedriver_path)
browser = webdriver.Chrome(service=webdriver_service, options=chrome_options)

# Abrir la página web
url = 'https://www.latamairlines.com/ec/es/ofertas-vuelos/?origin=UIO&outbound=2023-12-01T12%3A00%3A00.000Z&destination=MAD&inbound=null&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=ARRIVAL_DATE%2Casc'
browser.get(url)

# Esperar a que la página termine de cargar
delay = 10
try:
    vuelo = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="CenterWrapperBodyFlights"]/ol/li')))
    print('La página terminó de cargar')
    info_vuelos = obtener_info(browser)
except TimeoutException:
    print('La página tardó demasiado en cargar')
    info_vuelos = []

browser.quit()

info_vuelos


La página terminó de cargar
Se encontraron 15 vuelos.
Iniciando Scraping...


InvalidSelectorException: Message: invalid selector: Unable to locate an element with the xpath expression //*[@id="itinerary-modal-0-dialog-content"]/article/div/section[1]/div/[1]/div/div[2]/span[2] because of the following error:
SyntaxError: Failed to execute 'evaluate' on 'Document': The string '//*[@id="itinerary-modal-0-dialog-content"]/article/div/section[1]/div/[1]/div/div[2]/span[2]' is not a valid XPath expression.
  (Session info: chrome=118.0.5993.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
#0 0x562dffd53fb3 <unknown>
#1 0x562dffa274a7 <unknown>
#2 0x562dffa2c509 <unknown>
#3 0x562dffa2e120 <unknown>
#4 0x562dffa2e1dc <unknown>
#5 0x562dffa6eb06 <unknown>
#6 0x562dffa6eec1 <unknown>
#7 0x562dffa648a6 <unknown>
#8 0x562dffa9096d <unknown>
#9 0x562dffa64776 <unknown>
#10 0x562dffa90b0e <unknown>
#11 0x562dffaa9c02 <unknown>
#12 0x562dffa90713 <unknown>
#13 0